In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import hireverse.pipelines.cnn_preprocessing as cnn_preprocessing
from hireverse.utils.utils import BASE_DIR

participant_id = "P4"
frames = cnn_preprocessing.get_processed_frames(os.path.join(BASE_DIR, "data/raw/videos", f"{participant_id}.avi"), )

KeyboardInterrupt: 

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models

def build_competency_cnn(input_shape=(640, 640, 1), num_classes=600):
    model = models.Sequential()

    # Initial Convolutional Blocks
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.25))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.25))

    # Final Convolutional Layers
    model.add(layers.Conv2D(2048, (3, 3), activation='relu'))
    model.add(layers.GlobalAveragePooling2D())

    # Final Classification Layer
    model.add(layers.Dense(num_classes, activation='softmax'))  # 6 competencies × 100 classes

    # Compile with learning rate (as per paper)
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Initialize model
model = build_competency_cnn()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 638, 638, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 319, 319, 32)      0         
 D)                                                              
                                                                 
 batch_normalization (Batch  (None, 319, 319, 32)      128       
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 319, 319, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 317, 317, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 158, 158, 64)      0

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

# 1. Prepare Data
X = np.array([frame.image for frame in frames])  # Convert frames to numpy array
y = np.array(turker_scores_full_interview)      # Your labels

# Add channel dimension if grayscale
if X.ndim == 3:
    X = np.expand_dims(X, axis=-1)

# Split data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Create Data Generators
def create_data_generator(images, labels, batch_size=32, shuffle=True):
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(images))
    
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

batch_size = 32
train_dataset = create_data_generator(X_train, y_train, batch_size)
val_dataset = create_data_generator(X_val, y_val, batch_size, shuffle=False)

In [ ]:
# 3. Build and Train Model
model = build_competency_cnn()

epochs = 10
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs
)

In [ ]:
# 3. Build and Train Model
model = build_competency_cnn()

epochs = 10
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs
)